 PHASE 0: SETUP in Google Colab

In [1]:
!pip install requests python-dotenv


In [2]:
from google.colab import files

# Prompt user to upload .env file
uploaded = files.upload()


Saving .env.txt to .env.txt


In [14]:
import os

# Rename the uploaded file to '.env'
if os.path.exists(".env.txt"):
    os.rename(".env.txt", ".env")
    print("Renamed .env.txt to .env")
else:
    print("File not found. Please upload again.")


File not found. Please upload again.


In [15]:
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv('.env')

# Fetch and verify key
weather_key = os.getenv("OPENWEATHER_API")

if weather_key:
    print(" Weather API Key loaded successfully!")
    print("OPENWEATHER_API =", weather_key[:5] + "..." + weather_key[-5:])
else:
    print(" Failed to load API key.")


 Weather API Key loaded successfully!
OPENWEATHER_API = 576ba...00b85


In [16]:
class PlannerAgent:
    def __init__(self):
        pass

    def plan(self, goal: str):
        goal = goal.lower()
        steps = []

        if "launch" in goal or "spacex" in goal:
            steps.append("get_next_launch_info")

        if "weather" in goal:
            steps.append("check_weather_at_launch_site")

        if "delay" in goal or "predict" in goal:
            steps.append("predict_possible_delay")

        return steps


PHASE 1: Define the Goal & Build the Planner Agent

In [17]:
# Instantiate and test the planner
planner = PlannerAgent()

user_goal = "Find the next SpaceX launch, check the weather at that location, and tell if it may be delayed."
plan = planner.plan(user_goal)

print(" User Goal:", user_goal)
print(" Planner Output:", plan)


 User Goal: Find the next SpaceX launch, check the weather at that location, and tell if it may be delayed.
 Planner Output: ['get_next_launch_info', 'check_weather_at_launch_site', 'predict_possible_delay']


 PHASE 2.1: Launch Agent (SpaceX API)


In [18]:
import requests

class LaunchAgent:
    def __init__(self):
        self.base_url = "https://api.spacexdata.com/v4"

    def get_next_launch_info(self):
        try:
            # Step 1: Get next launch
            launch_url = f"{self.base_url}/launches/next"
            launch_data = requests.get(launch_url).json()

            rocket_id = launch_data.get("rocket")
            launchpad_id = launch_data.get("launchpad")
            launch_name = launch_data.get("name")
            launch_date = launch_data.get("date_utc")

            # Step 2: Get rocket name
            rocket_data = requests.get(f"{self.base_url}/rockets/{rocket_id}").json()
            rocket_name = rocket_data.get("name")

            # Step 3: Get launchpad details
            pad_data = requests.get(f"{self.base_url}/launchpads/{launchpad_id}").json()
            pad_name = pad_data.get("name")
            latitude = pad_data.get("latitude")
            longitude = pad_data.get("longitude")

            # Combine all info
            return {
                "launch_name": launch_name,
                "rocket": rocket_name,
                "launchpad": pad_name,
                "latitude": latitude,
                "longitude": longitude,
                "launch_date_utc": launch_date
            }

        except Exception as e:
            return {"error": str(e)}


In [20]:
launch_agent = LaunchAgent()
launch_info = launch_agent.get_next_launch_info()

print("Launch Info:")
for k, v in launch_info.items():
    print(f"{k}: {v}")


Launch Info:
launch_name: USSF-44
rocket: Falcon Heavy
launchpad: KSC LC 39A
latitude: 28.6080585
longitude: -80.6039558
launch_date_utc: 2022-11-01T13:41:00.000Z


 PHASE 2.2: Weather Agent (OpenWeatherMap API)


In [21]:
import os
import requests

class WeatherAgent:
    def __init__(self):
        self.api_key = os.getenv("OPENWEATHER_API")
        self.base_url = "https://api.openweathermap.org/data/2.5/weather"

    def get_weather(self, latitude, longitude):
        try:
            params = {
                "lat": latitude,
                "lon": longitude,
                "appid": self.api_key,
                "units": "metric"
            }

            response = requests.get(self.base_url, params=params)
            data = response.json()

            weather_info = {
                "location": f"{latitude}, {longitude}",
                "description": data['weather'][0]['description'],
                "temperature (°C)": data['main']['temp'],
                "wind_speed (m/s)": data['wind']['speed'],
                "cloudiness (%)": data['clouds']['all']
            }

            # Optional: include rain data if available
            if "rain" in data:
                weather_info["rain (mm)"] = data['rain'].get('1h', 0)

            return weather_info

        except Exception as e:
            return {"error": str(e)}


In [22]:
weather_agent = WeatherAgent()

# Use coordinates from previous step
latitude = 28.6080585
longitude = -80.6039558

weather = weather_agent.get_weather(latitude, longitude)

print("Weather Info:")
for k, v in weather.items():
    print(f"{k}: {v}")


Weather Info:
location: 28.6080585, -80.6039558
description: clear sky
temperature (°C): 25.09
wind_speed (m/s): 2.57
cloudiness (%): 0


PHASE 2.3: Delay Predictor Agent

In [23]:
class DelayPredictorAgent:
    def __init__(self):
        pass

    def predict(self, weather_info):
        reasons = []

        # Wind condition
        wind = weather_info.get("wind_speed (m/s)", 0)
        if wind > 10:
            reasons.append(f"High wind speed ({wind} m/s)")

        # Cloudiness
        clouds = weather_info.get("cloudiness (%)", 0)
        if clouds > 80:
            reasons.append(f"Heavy cloud cover ({clouds}%)")

        # Rain
        rain = weather_info.get("rain (mm)", 0)
        if rain > 0:
            reasons.append(f"Rain detected ({rain} mm)")

        if reasons:
            return {
                "delay_prediction": "Launch may be delayed",
                "reasons": reasons
            }
        else:
            return {
                "delay_prediction": "Launch unlikely to be delayed",
                "reasons": ["Weather conditions appear safe"]
            }


In [24]:
delay_agent = DelayPredictorAgent()

prediction = delay_agent.predict(weather)

print("Delay Prediction:")
print("Result:", prediction['delay_prediction'])
print("Reasons:")
for r in prediction['reasons']:
    print("-", r)


Delay Prediction:
Result: Launch unlikely to be delayed
Reasons:
- Weather conditions appear safe


PHASE 3: Orchestration & Chaining Logic

In [25]:
def run_goal(goal_text):
    print("User Goal:", goal_text)

    # Decide whether to use the class-based planner or simple functional one
    if "spacex" in goal_text.lower() or "launch" in goal_text.lower():
        planner = PlannerAgent()
        plan = planner.plan(goal_text)
    else:
        plan = simple_planner(goal_text)

    print("\n Planned Steps:", plan)

    # Intermediate storage
    context = {}

    # Run based on whether it's class-based or functional agent
    for step in plan:
        print(f"\n Executing step: {step}")

        if step == "get_next_launch_info":
            launch_agent = LaunchAgent()
            launch_info = launch_agent.get_next_launch_info()
            context["launch_info"] = launch_info
            print(" Launch Info:")
            for k, v in launch_info.items():
                print(f"  {k}: {v}")

        elif step == "check_weather_at_launch_site":
            weather_agent = WeatherAgent()
            coords = context.get("launch_info", {})
            if not coords:
                print("Launch info missing!")
                return

            weather = weather_agent.get_weather(coords['latitude'], coords['longitude'])
            context["weather_info"] = weather
            print("Weather Info:")
            for k, v in weather.items():
                print(f"  {k}: {v}")

        elif step == "predict_possible_delay":
            delay_agent = DelayPredictorAgent()
            weather = context.get("weather_info", {})
            if not weather:
                print("Weather info missing!")
                return

            prediction = delay_agent.predict(weather)
            context["final_prediction"] = prediction
            print("Delay Prediction Result:")
            print("  Result:", prediction["delay_prediction"])
            print("  Reasons:")
            for r in prediction["reasons"]:
                print("   -", r)

        elif step in agents:
            result = agents[step](context)
            context.update(result)

        else:
            print(f"Unknown step: {step}")

    return context.get("final_prediction", context)


In [26]:
run_goal("Find the next SpaceX launch, check the weather at that location, and tell if it may be delayed.")


User Goal: Find the next SpaceX launch, check the weather at that location, and tell if it may be delayed.

 Planned Steps: ['get_next_launch_info', 'check_weather_at_launch_site', 'predict_possible_delay']

 Executing step: get_next_launch_info
 Launch Info:
  launch_name: USSF-44
  rocket: Falcon Heavy
  launchpad: KSC LC 39A
  latitude: 28.6080585
  longitude: -80.6039558
  launch_date_utc: 2022-11-01T13:41:00.000Z

 Executing step: check_weather_at_launch_site
Weather Info:
  location: 28.6080585, -80.6039558
  description: clear sky
  temperature (°C): 25.09
  wind_speed (m/s): 2.57
  cloudiness (%): 0

 Executing step: predict_possible_delay
Delay Prediction Result:
  Result: Launch unlikely to be delayed
  Reasons:
   - Weather conditions appear safe


{'delay_prediction': 'Launch unlikely to be delayed',
 'reasons': ['Weather conditions appear safe']}

 PHASE 4: Evaluation & Logging

In [27]:
import json
from datetime import datetime

def evaluate_goal(goal_text, eval_id="eval_1"):
    print(f"\nRunning Evaluation: {eval_id}")
    result = run_goal(goal_text)

    log = {
        "evaluation_id": eval_id,
        "timestamp": datetime.utcnow().isoformat(),
        "goal": goal_text,
        "result": result
    }

    # Save to file
    with open(f"{eval_id}.json", "w") as f:
        json.dump(log, f, indent=4)

    print(f"\nEvaluation saved as '{eval_id}.json'")
    return log


In [28]:
evaluate_goal(
    "Find the next SpaceX launch, check the weather at that location, and tell if it may be delayed.",
    eval_id="eval_1_spacex_weather_delay"
)



Running Evaluation: eval_1_spacex_weather_delay
User Goal: Find the next SpaceX launch, check the weather at that location, and tell if it may be delayed.

 Planned Steps: ['get_next_launch_info', 'check_weather_at_launch_site', 'predict_possible_delay']

 Executing step: get_next_launch_info
 Launch Info:
  launch_name: USSF-44
  rocket: Falcon Heavy
  launchpad: KSC LC 39A
  latitude: 28.6080585
  longitude: -80.6039558
  launch_date_utc: 2022-11-01T13:41:00.000Z

 Executing step: check_weather_at_launch_site
Weather Info:
  location: 28.6080585, -80.6039558
  description: clear sky
  temperature (°C): 25.09
  wind_speed (m/s): 2.57
  cloudiness (%): 0

 Executing step: predict_possible_delay
Delay Prediction Result:
  Result: Launch unlikely to be delayed
  Reasons:
   - Weather conditions appear safe

Evaluation saved as 'eval_1_spacex_weather_delay.json'


{'evaluation_id': 'eval_1_spacex_weather_delay',
 'timestamp': '2025-06-16T11:30:48.925232',
 'goal': 'Find the next SpaceX launch, check the weather at that location, and tell if it may be delayed.',
 'result': {'delay_prediction': 'Launch unlikely to be delayed',
  'reasons': ['Weather conditions appear safe']}}

Phase 4.2 – Eval #2 (New Use Case)

In [29]:
from google.colab import files
import os
from dotenv import load_dotenv

# Step 1: Upload from local
print("📤 Please upload your `.env` file:")
uploaded = files.upload()

#  Step 2: Find any uploaded file that looks like .env
env_filename = None
for fname in uploaded.keys():
    if "env" in fname:
        env_filename = fname
        break

# Step 3: Rename to `.env`
if env_filename:
    os.rename(env_filename, ".env")
    print(f"Renamed '{env_filename}' to '.env'")
else:
    raise FileNotFoundError(".env file not found or not named correctly. Please try again.")

# Step 4: Load environment variables
load_dotenv('.env')

# Step 5: Fetch and verify NEWS_API_KEY
news_api_key = os.getenv("NEWS_API_KEY")

if news_api_key:
    print("News API Key loaded successfully!")
    print(" NEWS_API_KEY =", news_api_key[:5] + "..." + news_api_key[-5:])
else:
    print(" Failed to load NEWS_API_KEY. Please check your `.env` content.")


📤 Please upload your `.env` file:


Saving .env.txt to .env.txt
Renamed '.env.txt' to '.env'
News API Key loaded successfully!
 NEWS_API_KEY = d0edb...2ce05


In [30]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv('.env')

# Validate API key load
news_api_key = os.getenv("NEWS_API_KEY")
if news_api_key:
    print(" News API Key loaded:", news_api_key[:5] + "..." + news_api_key[-5:])
else:
    print(" NEWS_API_KEY not loaded.")


 News API Key loaded: d0edb...2ce05


In [31]:
import requests

def get_bitcoin_price(_):
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
    response = requests.get(url)
    data = response.json()
    price = data.get("bitcoin", {}).get("usd", "N/A")
    print(f" Bitcoin Price: ${price}")
    return {"bitcoin_price_usd": price}


In [32]:
def get_bitcoin_news(context):
    news_api_key = os.getenv("NEWS_API_KEY")
    url = f"https://newsapi.org/v2/everything?q=bitcoin&sortBy=publishedAt&apiKey={news_api_key}"

    response = requests.get(url)
    if response.status_code != 200:
        print(" Failed to fetch news:", response.status_code, response.text)
        return {"news_headlines": []}

    articles = response.json().get("articles", [])[:5]
    summaries = [article["title"] for article in articles]

    print("Bitcoin News Headlines:")
    for i, headline in enumerate(summaries, 1):
        print(f"  {i}. {headline}")

    return {"news_headlines": summaries}


In [33]:
def summarize_sentiment(context):
    headlines = context.get("news_headlines", [])
    positive_keywords = ["rise", "gain", "soar", "bullish", "profit"]
    negative_keywords = ["fall", "drop", "loss", "bearish", "crash"]

    score = 0
    for h in headlines:
        h = h.lower()
        score += sum(1 for word in positive_keywords if word in h)
        score -= sum(1 for word in negative_keywords if word in h)

    sentiment = " Neutral"
    if score > 1:
        sentiment = " Positive"
    elif score < -1:
        sentiment = " Negative"

    print(f" Market Sentiment: {sentiment}")
    return {"market_sentiment": sentiment}


In [34]:
def simple_planner(goal):
    goal_lower = goal.lower()
    if "spacex" in goal_lower:
        return ["get_next_launch_info", "check_weather_at_launch_site", "predict_possible_delay"]
    elif "bitcoin" in goal_lower:
        return ["get_bitcoin_price", "get_bitcoin_news", "summarize_sentiment"]
    return []


In [35]:
agents = {
    "get_bitcoin_price": get_bitcoin_price,
    "get_bitcoin_news": get_bitcoin_news,
    "summarize_sentiment": summarize_sentiment,
}


In [36]:
evaluate_goal(
    "Find the current price of Bitcoin, check recent news about Bitcoin, and summarize market sentiment.",
    eval_id="eval_2_bitcoin_sentiment"
)



Running Evaluation: eval_2_bitcoin_sentiment
User Goal: Find the current price of Bitcoin, check recent news about Bitcoin, and summarize market sentiment.

 Planned Steps: ['get_bitcoin_price', 'get_bitcoin_news', 'summarize_sentiment']

 Executing step: get_bitcoin_price
 Bitcoin Price: $106653

 Executing step: get_bitcoin_news
Bitcoin News Headlines:
  1. Bitcoin Investors Remain Calm Despite Israel-Iran Conflict Escalation
  2. Bitcoin price targets mushroom as traders bet on $140K+ this bull run
  3. Trump pocketed over $57 mn from crypto coin sales
  4. Seventh Presale Phase: Bitcoin Solaris Nears Public Launch with 10,000 TPS and 21M Supply Cap
  5. 10 Sunday Reads

 Executing step: summarize_sentiment
 Market Sentiment:  Neutral

Evaluation saved as 'eval_2_bitcoin_sentiment.json'


{'evaluation_id': 'eval_2_bitcoin_sentiment',
 'timestamp': '2025-06-16T11:32:20.760105',
 'goal': 'Find the current price of Bitcoin, check recent news about Bitcoin, and summarize market sentiment.',
 'result': {'bitcoin_price_usd': 106653,
  'news_headlines': ['Bitcoin Investors Remain Calm Despite Israel-Iran Conflict Escalation',
   'Bitcoin price targets mushroom as traders bet on $140K+ this bull run',
   'Trump pocketed over $57 mn from crypto coin sales',
   'Seventh Presale Phase: Bitcoin Solaris Nears Public Launch with 10,000 TPS and 21M Supply Cap',
   '10 Sunday Reads'],
  'market_sentiment': ' Neutral'}}

In [37]:
!ls


eval_1_spacex_weather_delay.json  eval_2_bitcoin_sentiment.json  sample_data


FastAPI Endpoint for Evaluation + Storage


In [38]:
# STEP 1: Install necessary libraries
!pip install fastapi uvicorn nest_asyncio pyngrok python-multipart aiofiles --quiet


In [39]:
#  Configure ngrok authtoken
!ngrok config add-authtoken 2yMN4GMKS6s6c4qH5iSCm2ieiIR_3KGhaEMmUbndyr7ZjVPe4


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [40]:
# STEP 1: Install required packages
!pip install fastapi uvicorn nest_asyncio pyngrok python-multipart aiofiles python-dotenv --quiet


In [ ]:

# STEP 2: Imports
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List
import os
import json
from datetime import datetime
import requests
from dotenv import load_dotenv
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# STEP 3: Load environment variables from uploaded .env file
uploaded = files.upload()  # You must upload your `.env.txt` file here
env_file_path = next(iter(uploaded))  # get uploaded filename
if not env_file_path.endswith('.txt'):
    raise ValueError("Uploaded file must be `.env.txt` format.")
os.rename(env_file_path, '.env')  # rename to .env
load_dotenv('.env')

#  STEP 4: Define Agents
def get_bitcoin_price(_):
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
    data = requests.get(url).json()
    return {"bitcoin_price_usd": data.get("bitcoin", {}).get("usd", "N/A")}

def get_bitcoin_news(context):
    api_key = os.getenv("NEWS_API_KEY")
    url = f"https://newsapi.org/v2/everything?q=bitcoin&sortBy=publishedAt&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code != 200:
        return {"news_headlines": []}
    articles = response.json().get("articles", [])[:5]
    return {"news_headlines": [a["title"] for a in articles]}

def summarize_sentiment(context):
    headlines = context.get("news_headlines", [])
    pos = ["rise", "gain", "soar", "bullish", "profit"]
    neg = ["fall", "drop", "loss", "bearish", "crash"]
    score = sum(any(p in h.lower() for p in pos) for h in headlines) - \
            sum(any(n in h.lower() for n in neg) for h in headlines)
    if score > 1: return {"market_sentiment": " Positive"}
    elif score < -1: return {"market_sentiment": " Negative"}
    return {"market_sentiment": " Neutral"}

def get_next_launch_info(_):
    data = requests.get("https://api.spacexdata.com/v4/launches/next").json()
    return {
        "mission": data.get("name"),
        "launch_date": data.get("date_utc"),
        "launchpad": data.get("launchpad"),
    }

def check_weather_at_launch_site(_):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat=28.56&lon=-80.57&appid={os.getenv('OPENWEATHER_API_KEY')}&units=metric"
    r = requests.get(url)
    if r.status_code != 200:
        return {"weather": "Unavailable"}
    data = r.json()
    return {
        "temperature": data["main"]["temp"],
        "wind_speed": data["wind"]["speed"],
        "description": data["weather"][0]["description"]
    }

def predict_possible_delay(context):
    wind = context.get("wind_speed", 0)
    temp = context.get("temperature", 0)
    if wind > 15 or temp < 5:
        return {"launch_delay_risk": " High"}
    return {"launch_delay_risk": " Low"}

#  STEP 5: Planner + Evaluator
def simple_planner(goal):
    g = goal.lower()
    if "bitcoin" in g:
        return ["get_bitcoin_price", "get_bitcoin_news", "summarize_sentiment"]
    elif "spacex" in g or "launch" in g:
        return ["get_next_launch_info", "check_weather_at_launch_site", "predict_possible_delay"]
    return []

def run_goal(goal):
    plan = simple_planner(goal)
    context = {}
    for step in plan:
        fn = agents.get(step)
        if fn:
            context.update(fn(context))
    return context

def evaluate_goal(goal, eval_id=None):
    result = run_goal(goal)
    data = {
        "evaluation_id": eval_id or f"eval_{datetime.utcnow().isoformat()}",
        "timestamp": datetime.utcnow().isoformat(),
        "goal": goal,
        "result": result
    }
    os.makedirs("evaluations", exist_ok=True)
    with open(f"evaluations/{data['evaluation_id']}.json", "w") as f:
        json.dump(data, f, indent=2)
    return data

#  STEP 6: Agent Map
agents = {
    "get_bitcoin_price": get_bitcoin_price,
    "get_bitcoin_news": get_bitcoin_news,
    "summarize_sentiment": summarize_sentiment,
    "get_next_launch_info": get_next_launch_info,
    "check_weather_at_launch_site": check_weather_at_launch_site,
    "predict_possible_delay": predict_possible_delay,
}

# STEP 7: FastAPI App
app = FastAPI()

class GoalRequest(BaseModel):
    goal: str
    eval_id: str = None

@app.post("/evaluate")
def evaluate(request: GoalRequest):
    output = evaluate_goal(request.goal, request.eval_id)
    return JSONResponse(content=output)

@app.get("/evaluations")
def list_evaluations():
    results = []
    for file in os.listdir("evaluations"):
        if file.endswith(".json"):
            with open(f"evaluations/{file}") as f:
                results.append(json.load(f))
    return results

#  STEP 8: Run FastAPI inside notebook using uvicorn programmatically
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"🚀 FastAPI running at: {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)


Saving .env.txt to .env.txt


INFO:     Started server process [625]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 FastAPI running at: NgrokTunnel: "https://b660-35-233-131-53.ngrok-free.app" -> "http://localhost:8000"
INFO:     110.226.90.233:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     110.226.90.233:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     110.226.90.233:0 - "POST /evaluate HTTP/1.1" 200 OK


In [ ]:
#Access WebPage here: https://b660-35-233-131-53.ngrok-free.app/docs
